In [ ]:
#bits of tips from here and there while i'm trying to learn the "proper" or the "better" ways to analyze datasets.


In [ ]:
#1 Questioning the Question
"""“What kinds of data are available?”; 
“What action might result in answering this question?”; “What resources are available to do this analysis?”; .
“Is it possible to collect new data?”"""


In [1]:
#EDA

#missing data function
def missing_data(data):
    total = data.isnull().sum().sort_values(ascending = False)
    percent = (data.isnull().sum()/data.isnull().count()*100).sort_values(ascending = False)
    return pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])


#null check
data.isnull().sum()

#describe data
data.describe()
#nunique 
data.describe(include =['O'])
data.nunique()

#crosstab for stats
pd.crosstab(df_train.Pclass, df_train.Survived, margins=True).style.background_gradient(cmap='autumn_r')
pd.crosstab(df.A,df.B, normalize='index')

#seaborn hist graph comparing a var with true outcomes/classification
import seaborn as sns
%matplotlib inline
g = sns.FacetGrid(data, col = 'Attrition_cat')
g.map(plt.hist,'JobSatisfaction', bins = 5)

#other way to do plot above
sns.catplot(x='Sex', col='Survived', kind='count', data=df_train);
#for the catplot, you can switch out var like below for a different take 
sns.catplot(x='Survived', col='Embarked', kind='count', data=df_train);
#or
sns.catplot('Embarked','Survived', hue= 'Sex', kind='point', data=df_train)
plt.show()

#and get the %
print("% of women survived: " , df_train[df_train.Sex == 'female'].Survived.sum()/df_train[df_train.Sex == 'female'].Survived.count())
print("% of men survived:   " , df_train[df_train.Sex == 'male'].Survived.sum()/df_train[df_train.Sex == 'male'].Survived.count())

#pairplots
col = ['Attrition', 'Age', 'MonthlyIncome','JobLevel','DistanceFromHome']
sns.pairplot(data[col], kind = 'reg', diag_kind = 'kde', hue = 'Attrition')

#.cat.codes (similar to get dummies)
data["Attrition_cat"] = data["Attrition"].cat.codes
#I used cat.codes to add a new column called attrition_cat as below. Cool!

###countplot This could lead to either under sampling or not since we can find unbalance-ness in train and test datasets
sns.countplot(x='Survived', data=df_train);
sns.countplot(x='Survived', data=df_test);

#% of ppl survived in train dataset.
print(df_train.AMI_FLAG.sum()/df_test.AMI_FLAG.count())

#group_by categories 
df_train.groupby(['Survived','Sex'])['Survived'].count()


NameError: name 'data' is not defined

In [ ]:
#Wrangling
#correlation maxtrix and heatmap visualization
corr = df_train_ml.corr()

f,ax = plt.subplots(figsize=(9,6))
sns.heatmap(corr, annot = True, linewidths=1.5 , fmt = '.2f',ax=ax)
plt.show()

#get dummies, drop columns, dropnas
df_train_ml = pd.get_dummies(df_train_ml, columns=['Sex', 'Embarked', 'Pclass'], drop_first=True)
df_train_ml.drop(['PassengerId','Name','Ticket', 'Cabin', 'Age_bin', 'Fare_bin'],axis=1,inplace=True)
df_train_ml.dropna(inplace=True)

#Dealing with missing data:
1. StringIO(), isnull() to find null data. dropna()-axis, how, thresh, subset to get rid of NaN
2. Interpolation: Imputer using mean, median or most_frequent. sklearn.preprocessing/Imputer
3. Categorical data: mapping or encode class labels. Or easier: pd.get_dummies


#Partitioning Data:
1. sklearn.crosslidation import train_test_split (refer to classification exercise for example). Assigns test data size need balancing.

#Standardize the features (putting em on the same scale):
1. Normalization: min-max scaling (x - min / range). 
sklearn.preprocessing import MinMaxScaler
mms = MinMaxScaler()
x_train_norm = mms.fit_transform(X_Train)
X_test_norm = mms.transform(X_Test)

#2. Standardization: statistical method to find Z
code similar to MinMaxScaler.

Select Meaningful features: some ways to prevent overfitting: collect more data, intro penalty for complexity via regularization, 
simpler model, reduce attributes/dimensionality of data
Penalty regularization: L1 and L2 sklearn.linear LogisticRegression

In [1]:
#Feature selection: Selection and Extraction
page 120 Seb's book on Sequential Backward Selection to help you visualize and choose best features.
PAge 124 will talk about using random forest to calculate and visualize feature importance (no need standardized or normalized)

#Correlation analysis between continuous variable and target variable:
#correlation maxtrix and heatmap visualization
corr = df_train_ml.corr()

f,ax = plt.subplots(figsize=(9,6))
sns.heatmap(corr, annot = True, linewidths=1.5 , fmt = '.2f',ax=ax)
plt.show()

#Feature selection: analyzing categorical variables vs target categorical variable with Chi Square test:
#keep in mind: Null hypothesis Ho = NO correlation between var and target categorical var.
class ChiSquare:
    def __init__(self, dataframe):
        self.df = dataframe
        self.p = None #P-Value
        self.chi2 = None #Chi Test Statistic
        self.dof = None
        
        self.dfObserved = None
        self.dfExpected = None
        
    def _print_chisquare_result(self, colX, alpha):
        result = ""
        if self.p<alpha:
            result="{0} is IMPORTANT for Prediction".format(colX)
        else:
            result="{0} is NOT an important predictor. (Discard {0} from model)".format(colX)

        print(result)
        
    def TestIndependence(self,colX,colY, alpha=0.05):
        X = self.df[colX].astype(str)
        Y = self.df[colY].astype(str)
        
        self.dfObserved = pd.crosstab(Y,X) 
        chi2, p, dof, expected = stats.chi2_contingency(self.dfObserved.values)
        self.p = p
        self.chi2 = chi2
        self.dof = dof 
        
        self.dfExpected = pd.DataFrame(expected, columns=self.dfObserved.columns, index = self.dfObserved.index)
        
        self._print_chisquare_result(colX,alpha)
#Initialize ChiSquare Class
cT = ChiSquare(df)

#Feature Selection
testColumns = ['ORIG_REAS_ENTITLE_CD','Education_level','Online_purchaser','Est_BMI_decile']
for var in testColumns:
    cT.TestIndependence(colX=var,colY="AMI_FLAG")
    





SyntaxError: invalid syntax (<ipython-input-1-bb98acbd6b36>, line 3)

In [ ]:
#From loading to submission:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
PATH = "Oren/Kaggle/Housing Prices/"  #where you put the files
df_train = pd.read_csv(f'{PATH}train.csv', index_col='Id')
df_test = pd.read_csv(f'{PATH}test.csv', index_col='Id')
target = df_train['SalePrice']  #target variable
df_train = df_train.drop('SalePrice', axis=1)
df_train['training_set'] = True
df_test['training_set'] = False
df_full = pd.concat([df_train, df_test])
df_full = df_full.interpolate()
df_full = pd.get_dummies(df_full)
df_train = df_full[df_full['training_set']==True]
df_train = df_train.drop('training_set', axis=1)
df_test = df_full[df_full['training_set']==False]
df_test = df_test.drop('training_set', axis=1)
rf = RandomForestRegressor(n_estimators=100, n_jobs=-1)
rf.fit(df_train, target)
preds = rf.predict(df_test)
my_submission = pd.DataFrame({'Id': df_test.index, 'SalePrice': preds})
my_submission.to_csv(f'{PATH}submission.csv', index=False)